# Data exploration and descriptive statistics of satellite imagery

This jupyter notebook will parse, collate and analyze satellite imagery files from Maxar (former DigitalGlobe) The parsing of the metadata files (`DeliveryMetadata.xml`) are mainly general and should work for any Maxar product delivery. After parsing of the metadata files are converted to pandas dataframes the code is mainly project specific.

## Imports and setup

In [ ]:
import numpy as np
import pandas as pd
import os
from modules.metadata_reader import img_metadata_to_dict, add_names_to_metadata_dict, dict_to_df

# Path to location where individual satellite images are located
DATA_PATH = 'data' 
DATA_PATH_IS_RELATIVE = True

# Name of metadata .xml file
METADATA_NAME = 'DeliveryMetadata.xml'

# Names of areas covered by satellite imagery
AREAS = ['La_Spezia', 'Toulon'] # Spelled like the directory names

# Speficy what the xmlns url on top of metadata .xml file is
# (should be second line)
XMLNS = 'http://xsd.digitalglobe.com/xsd/dm'

## Metadata parsing from xml to pandas dataframe

Every satellite image delivery from Maxar contains a `DeliveryMetadata.xml` file with important specifications for both the multispectral and panchromatic images. The following functions finds all the `DeliveryMetadata.xml` files contained in all subdirectories of a directory and parses them into the *Pandas DataFrame* format which will be used for further descriptive statistics of the dataset.

In [ ]:
img_metadata_pan, img_metadata_ms = img_metadata_to_dict(METADATA_NAME, 
                                                         DATA_PATH, XMLNS, 
                                                         path_is_relative = DATA_PATH_IS_RELATIVE)

img_metadata_pan = add_names_to_metadata_dict(img_metadata_pan, AREAS)
img_metadata_ms = add_names_to_metadata_dict(img_metadata_ms, AREAS)

img_metadata_pan = dict_to_df(img_metadata_pan)
img_metadata_ms = dict_to_df(img_metadata_ms)

# Distribution of images

## Some quick sanity checks

In [ ]:
print('Number of panchromatic images:', len(img_metadata_pan))
print('Number of multispectral images:', len(img_metadata_ms))
print()
if list(img_metadata_pan.index.values) == list(img_metadata_ms.index.values):
    print('Pass: Identical keys. Keys in panchromatic and multispectral dictionaries are identical.')
else: print('Fail: Identical keys. Keys in panchromatic and multispectral dictionaries are not identical.')
print()

## Histograms multispectral

In [ ]:
hist_factors = ['pixelWidth', 
                'pixelHeight', 
                'n_bands',
                'offNadirAngle', 
                'sunAzimuth', 
                'sunElevation']
hist_ms = img_metadata_ms[hist_factors].hist(figsize=(15,8))

In [ ]:
hist_pan = img_metadata_pan[hist_factors].hist(figsize=(15,8))

## Contingency table Sensor x Area

In [ ]:
table = pd.crosstab(img_metadata_pan['area_name'], 
                    img_metadata_pan['sensorVehicle'])
table

## Clouds?
*TODO: Get a measure of clod coverage in every specific image. Either automatically based on available algorithms or manually since the number of images are limited.*

## Calculate actual area covered by each image
*TODO: Many images do not cover the whole area of interest. By using the parsed metadata dataframe and shape geometry supplied with every image it is possible to calculate the actual area. This number may nuance distribution of images across areas, sensors and clouds.*